### Домашнее задание 2. Часть 1.

In [9]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter
import time
import pandas as pd

# Загружаем необходимые ресурсы NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/artembritsyn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/artembritsyn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/artembritsyn/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/artembritsyn/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

- 1.Рассчитайте метрики TF-IDF для любых 3 песен на одном языке, которые вы сами
выберите. Не забудьте, что нужно привести слова к начальной форме, убрать
стоп-слова.

In [10]:
def preprocess_text(text):
    # Токенизация
    tokens = word_tokenize(text.lower())
    
    # Удаление стоп-слов и пунктуации
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    
    # Лемматизация
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return ' '.join(tokens)

# Читаем тексты песен
with open('lyrics/lyrics_1.txt', 'r') as f:
    song1 = f.read()
with open('lyrics/lyrics_2.txt', 'r') as f:
    song2 = f.read()
with open('lyrics/lyrics_3.txt', 'r') as f:
    song3 = f.read()

# Предобработка текстов
songs = [song1, song2, song3]
processed_songs = [preprocess_text(song) for song in songs]

# 1. TF-IDF анализ
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(processed_songs)
feature_names = tfidf.get_feature_names_out()


 - 2. Сравните его с другим методам векторизации текста, например, с Count Vectorizer,
Word2Vec или Doc2Vec. Необходимо сделать вывод приносит ли TF-IDF улучшения по
сравнению с альтернативными подходами, с точки зрения вычислительной
эффективности и масштабируемости алгоритма при обработке больших объемов
данных разных данных.

In [11]:
# 2. Count Vectorizer для сравнения
count_vec = CountVectorizer()
count_matrix = count_vec.fit_transform(processed_songs)

# Анализ времени выполнения
start_time = time.time()
tfidf.fit_transform(processed_songs)
tfidf_time = time.time() - start_time

start_time = time.time()
count_vec.fit_transform(processed_songs)
count_time = time.time() - start_time


print(f"\nВремя выполнения TF-IDF: {tfidf_time:.4f} секунд")
print(f"Время выполнения Count Vectorizer: {count_time:.4f} секунд")


Время выполнения TF-IDF: 0.0029 секунд
Время выполнения Count Vectorizer: 0.0012 секунд


* TF-IDF более эффективен для определения важности слов, так как учитывает контекст всего корпуса текстов
* Count Vectorizer проще и быстрее, но дает менее информативные результаты

С точки зрения масштабируемости:
TF-IDF требует больше вычислительных ресурсов, а
Count Vectorizer более эффективен при обработке больших объемов данных, однако оба метода хорошо работают с разреженными матрицами

- 3. Сделайте исследование по полученнм преобразованным данным.
Какие слова/слово сочения чаще всего встречаются, а какие реже.

In [12]:
# 3. Анализ частоты слов
def get_top_words(matrix, feature_names, n=10):
    sums = matrix.sum(axis=0).A1
    top_indices = sums.argsort()[-n:][::-1]
    return [(feature_names[i], sums[i]) for i in top_indices]

# Получаем топ слова для каждого метода
tfidf_top = get_top_words(tfidf_matrix, feature_names)
count_top = get_top_words(count_matrix, count_vec.get_feature_names_out())

# Выводим результаты
print("=== TF-IDF Топ 10 слов ===")
for word, score in tfidf_top:
    print(f"{word}: {score:.4f}")

print("\n=== Count Vectorizer Топ 10 слов ===")
for word, count in count_top:
    print(f"{word}: {int(count)}")

=== TF-IDF Топ 10 слов ===
hole: 0.5767
land: 0.4471
bottle: 0.4267
yeah: 0.4166
message: 0.3879
hope: 0.3879
someone: 0.3491
take: 0.3478
get: 0.3033
soul: 0.2827

=== Count Vectorizer Топ 10 слов ===
yeah: 13
bottle: 11
hole: 10
hope: 10
message: 10
get: 10
someone: 9
oh: 9
land: 9
soul: 7


### Часть 2
1. Скачайте датасет отзывов на фильмы. Датасет содержит текст отзыва и бинарную
метку тональности (положительный/отрицательный).
2. Используйте библиотеку Hugging Face для загрузки предварительно обученной
модели BERT и токенизатора.
3. Подготовьте данные: используйте токенизатор BERT для преобразования текстовых
данных в формат , который можно подать на вход модели BERT .
4. Создайте классификатор на основе BERT : это может быть модель BERT с одним
линейным слоем для классификации на вершине.
5. Обучите классификатор на данных обучения и оцените его производительность на
данных для тестирования.

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd

In [18]:
# 1
df = pd.read_csv("IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [25]:
train_texts, test_texts, train_sentiments, test_sentiments = train_test_split(
    df["review"].tolist(), df["sentiment"].tolist(), test_size=0.2, random_state=42
)

In [21]:
# 2
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)


/Users/artembritsyn/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [26]:
#Сделаем класс датасет на торче
class MovieReviewDataset(Dataset):
    def __init__(self, encodings, sentiments):
        self.encodings = encodings
        self.sentiments = sentiments

    def __len__(self):
        return len(self.sentiments)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["sentiments"] = torch.tensor(self.sentiments[idx])
        return item

train_dataset = MovieReviewDataset(train_encodings, train_sentiments)
test_dataset = MovieReviewDataset(test_encodings, test_sentiments)


In [23]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [27]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

optimizer = AdamW(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()

/Users/artembritsyn/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [28]:
EPOCHS = 3
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")


TypeError: new(): invalid data type 'str'